In [ ]:
import os
import cv2
import src.preprocessing as ppc
from tqdm.notebook import tqdm
import json
import math

## Run this

In [ ]:
with open('original_shuffled_map.json') as fp:
    original_shuffled_map = json.load(fp)

In [ ]:
CUTOFF_THRESHOLD = 0
RESOLUTION = 416
OUT_PATH = f'data/shuffled_square_png_{RESOLUTION}'

if not os.path.isdir(OUT_PATH):
    print(f'{OUT_PATH} does not exist, creating dir')
    os.mkdir(OUT_PATH)

In [ ]:
from multiprocessing import Pool

def _make_shuffled_square_img(filename):
    try:
        file = f"data/cfp/{filename}"
        img = cv2.imread(file)
        square_img, scaling, cutting, padding = ppc.square_resize(img, RESOLUTION, CUTOFF_THRESHOLD)
                
        new_name = original_shuffled_map[filename]
        cv2.imwrite(f"{OUT_PATH}/{new_name}.png", square_img)
        return (filename, new_name, cutting, padding, square_img.shape[0], scaling, img.shape)

    except Exception as e:
        print(filename, e)
        
        
        
l_files = os.listdir("data/cfp")

In [ ]:
with Pool() as pool:
    op_metadata = list(tqdm(pool.imap(_make_shuffled_square_img, l_files), total=len(l_files)))    
print('Finished.')

In [ ]:
import pandas as pd

df_img_info = pd.DataFrame.from_records([{
        'orig_file' : opdata[0],
        'new_file' : opdata[1],
        'delta_x' : opdata[3][0] - opdata[2][0],
        'delta_y' : opdata[3][2] - opdata[2][2],
        'orig_crop_side' : round(opdata[4] * opdata[5]),
        'side' : RESOLUTION,
        'scaling' : round(opdata[4] * opdata[5]) / RESOLUTION,
    } for opdata in op_metadata if opdata is not None])

df_img_info.to_csv(os.path.join(OUT_PATH, 'img_info.csv'))
df_img_info